## 1. Load Data

In [1]:
import os

# Read Folder
folder_path = '..\DataSet_03'
# Read File in Folder
file_names = os.listdir(folder_path)

# Calculate
total_images = len([file for file in file_names if file.endswith(('.png', '.jpg', '.jpeg'))])

# Print File
print(f'Total gambar dalam folder: {total_images}\n\nNama file setiap data:')
for file_name in file_names:
    print(file_name)


Total gambar dalam folder: 7543

Nama file setiap data:
0_DataSet3_001.jpg
0_DataSet3_002.jpg
0_DataSet3_003.jpg
0_DataSet3_004.jpg
0_DataSet3_005.jpg
0_DataSet3_006.jpg
0_DataSet3_007.jpg
0_DataSet3_008.jpg
0_DataSet3_009.jpg
0_DataSet3_010.jpg
0_DataSet3_011.jpg
0_DataSet3_012.jpg
0_DataSet3_013.jpg
0_DataSet3_014.jpg
0_DataSet3_015.jpg
0_DataSet3_016.jpg
0_DataSet3_017.jpg
0_DataSet3_018.jpg
0_DataSet3_019.jpg
0_DataSet3_020.jpg
0_DataSet3_021.jpg
0_DataSet3_022.jpg
0_DataSet3_023.jpg
0_DataSet3_024.jpg
0_DataSet3_025.jpg
0_DataSet3_026.jpg
0_DataSet3_027.jpg
0_DataSet3_028.jpg
0_DataSet3_029.jpg
0_DataSet3_030.jpg
0_DataSet3_031.jpg
0_DataSet3_032.jpg
0_DataSet3_033.jpg
0_DataSet3_034.jpg
0_DataSet3_035.jpg
0_DataSet3_036.jpg
0_DataSet3_037.jpg
0_DataSet3_038.jpg
0_DataSet3_039.jpg
0_DataSet3_040.jpg
0_DataSet3_041.jpg
0_DataSet3_042.jpg
0_DataSet3_043.jpg
0_DataSet3_044.jpg
0_DataSet3_045.jpg
0_DataSet3_046.jpg
0_DataSet3_047.jpg
0_DataSet3_048.jpg
0_DataSet3_049.jpg
0_DataSet3_05

In [2]:
import numpy as np

# Random File
np.random.shuffle(file_names)

# Ratio Training
train_ratios = [0.6, 0.7, 0.8]  # Proporsi 60:40, 70:30, 80:20

for train_ratio in train_ratios:
    train_size = int(len(file_names) * train_ratio)

    # Spliting File
    train_file_names = file_names[:train_size]
    test_file_names = file_names[train_size:]

    # Print Test and Training Data
    print(f'\nSplit Data Latih {int(train_ratio * 100)}%')
    print(f'Total Data Latih: {len(train_file_names)}')
    print(f'Total Data Uji: {len(test_file_names)}')

    # Create Label
    train_labels = [int(file.split('_')[0]) for file in train_file_names]
    test_labels = [int(file.split('_')[0]) for file in test_file_names]


Split Data Latih 60%
Total Data Latih: 4525
Total Data Uji: 3018

Split Data Latih 70%
Total Data Latih: 5280
Total Data Uji: 2263

Split Data Latih 80%
Total Data Latih: 6034
Total Data Uji: 1509


In [3]:
from collections import Counter

# Create a dictionary to store the number of occurrences of each number
digit_counts = Counter()

for file_name in file_names:
    first_digit = file_name[0]
    digit_counts[first_digit] += 1

# Print the number of occurrences of each number
for digit, count in digit_counts.items():
    print(f'Angka {digit}: {count} gambar')


Angka 8: 751 gambar
Angka 0: 751 gambar
Angka 7: 751 gambar
Angka 4: 751 gambar
Angka 3: 751 gambar
Angka 6: 751 gambar
Angka 1: 778 gambar
Angka 5: 751 gambar
Angka 9: 757 gambar
Angka 2: 751 gambar


## 2. Image Preprocessing

In [4]:
from PIL import Image
import numpy as np

def normalize_image(image_path, target_size=(128, 128)):
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = image.resize(target_size)  # Resize the image
    image_np = np.array(image) / 255.0  # Rescale pixel values to [0, 1]
    return image_np

# Normalize Images
normalized_images = []
labels = []

for file_name in file_names:
    if file_name.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, file_name)
        normalized_image = normalize_image(image_path)
        normalized_images.append(normalized_image)
        labels.append(int(file_name.split('_')[0]))


# Converts list of images and labels to numpy array
normalized_images = np.array(normalized_images)
labels = np.array(labels)

print(f'Shape Images: {normalized_images.shape}')
print(f'Shape Labels: {labels.shape}')


Shape Images: (7543, 128, 128)
Shape Labels: (7543,)


## 3. Model Implementation

### For Spliting Train 60% and Test 40%

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Standardization
scaler = StandardScaler()
num_images, height, width = normalized_images.shape
standar_images = scaler.fit_transform(normalized_images.reshape(-1, height * width))

# Spliting
test_ratio = 0.4
X_train, X_test, y_train, y_test  = train_test_split(standar_images, labels, test_size=test_ratio, random_state=42)

X_train_flat = np.array([img.flatten() for img in X_train])
X_test_flat = np.array([img.flatten() for img in X_test])

# Model KNN
knn = KNeighborsClassifier(n_neighbors=3) 
knn.fit(X_train_flat, y_train)

# Prediction
y_pred = knn.predict(X_test_flat)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi Model KNN tanpa ekstraksi fitur untuk Data Test {int((test_ratio) * 100)}%: {accuracy * 100:.2f}%')

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Akurasi Model KNN tanpa ekstraksi fitur untuk Data Test 40%: 41.88%

Classification Report:
              precision    recall  f1-score   support

           0       0.47      0.60      0.53       300
           1       0.44      0.86      0.58       298
           2       0.41      0.50      0.45       313
           3       0.25      0.38      0.30       299
           4       0.48      0.61      0.54       290
           5       0.37      0.33      0.35       298
           6       0.52      0.25      0.33       330
           7       0.62      0.40      0.49       299
           8       0.33      0.13      0.18       289
           9       0.37      0.14      0.20       302

    accuracy                           0.42      3018
   macro avg       0.43      0.42      0.40      3018
weighted avg       0.43      0.42      0.40      3018



### For Spliting Train 70% and Test 30%

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Standardization
scaler = StandardScaler()
num_images, height, width = normalized_images.shape
standar_images = scaler.fit_transform(normalized_images.reshape(-1, height * width))

# Spliting
test_ratio = 0.3
X_train, X_test, y_train, y_test  = train_test_split(standar_images, labels, test_size=test_ratio, random_state=42)

X_train_flat = np.array([img.flatten() for img in X_train])
X_test_flat = np.array([img.flatten() for img in X_test])

# Model KNN
knn = KNeighborsClassifier(n_neighbors=3) 
knn.fit(X_train_flat, y_train)

# Prediction
y_pred = knn.predict(X_test_flat)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi Model KNN tanpa ekstraksi fitur untuk Data Test {int((test_ratio) * 100)}%: {accuracy * 100:.2f}%')

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Akurasi Model KNN tanpa ekstraksi fitur untuk Data Test 30%: 43.75%

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.67      0.57       219
           1       0.46      0.90      0.61       231
           2       0.44      0.52      0.47       242
           3       0.25      0.40      0.31       217
           4       0.52      0.63      0.57       223
           5       0.41      0.31      0.35       232
           6       0.49      0.25      0.33       236
           7       0.66      0.42      0.52       222
           8       0.38      0.15      0.21       217
           9       0.31      0.12      0.17       224

    accuracy                           0.44      2263
   macro avg       0.44      0.44      0.41      2263
weighted avg       0.44      0.44      0.41      2263



### For Spliting Train 80% and Test 20%

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Standardization
scaler = StandardScaler()
num_images, height, width = normalized_images.shape
standar_images = scaler.fit_transform(normalized_images.reshape(-1, height * width))

# Spliting
test_ratio = 0.2
X_train, X_test, y_train, y_test  = train_test_split(standar_images, labels, test_size=test_ratio, random_state=42)

X_train_flat = np.array([img.flatten() for img in X_train])
X_test_flat = np.array([img.flatten() for img in X_test])

# Model KNN
knn = KNeighborsClassifier(n_neighbors=3) 
knn.fit(X_train_flat, y_train)

# Prediction
y_pred = knn.predict(X_test_flat)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi Model KNN tanpa ekstraksi fitur untuk Data Test {int((test_ratio) * 100)}%: {accuracy * 100:.2f}%')

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Akurasi Model KNN tanpa ekstraksi fitur untuk Data Test 20%: 44.47%

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.67      0.57       150
           1       0.48      0.89      0.63       161
           2       0.44      0.55      0.49       155
           3       0.25      0.41      0.31       145
           4       0.51      0.64      0.57       144
           5       0.42      0.30      0.35       155
           6       0.52      0.24      0.33       169
           7       0.63      0.43      0.51       136
           8       0.41      0.17      0.24       138
           9       0.36      0.13      0.19       156

    accuracy                           0.44      1509
   macro avg       0.45      0.44      0.42      1509
weighted avg       0.45      0.44      0.42      1509

